<a href="https://colab.research.google.com/github/Clarice-Satiko-Aoto/Analise_dados_b/blob/WIP/vendas_feat_e_eda4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install workalendar

In [245]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from workalendar.america import Brazil

# Crie um objeto de calendário brasileiro que exclui os feriados.
cal = Brazil()

In [246]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Carregando os dados

In [247]:
df_geral = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/vendas/vendas.csv',
                 parse_dates=['data'])


# #Meta 2022
meta22 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/vendas/data/Meta_2022.csv')

#Meta 2023
# mapeando o caminho
arquivo_excel = pd.ExcelFile('/content/drive/MyDrive/Colab Notebooks/vendas/data/Meta_2023.xlsx')
#lendo todas as tabelas
planilhas = arquivo_excel.parse(sheet_name=None,
                                header=2,
                                usecols=['Categoria Produto', 'Previsão Vendas', '1', '2', '3','4'])
#armazenando as planilhas
victor1_2023o = planilhas['1 - Victor Castro']
gabriel2_2023o = planilhas['2 - Gabriel Azevedo']


#De Para
#Supervisor
df_supervisor = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/vendas/data/PBSupervisor.csv')

#Vendedor
df_vendedor = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/vendas/data/PBVendedor.csv')

### Funções

In [277]:
def dias_uteis_sem_feriado(df):
  data_inicial = df.index.min()
  data_final = df.index.max()

  # Gere um índice de datas diárias.
  datas_diarias = pd.date_range(start=data_inicial, end=data_final, freq='D')

  # Filtrar as datas úteis, excluindo fins de semana e feriados brasileiros.
  dias_sem_feriado = [data for data in datas_diarias if data.weekday() < 5 and data not in cal.holidays()]
  return dias_sem_feriado

In [249]:
# Função para calcular a quantidade de dias úteis em um mês específico.

def calcular_dias_uteis_no_mes(row):
    """
    calcular os dias uteis no mes
    """
    ano, mes = row.name.year, row.name.month
    primeiro_dia = pd.Timestamp(year=ano, month=mes, day=1)
    ultimo_dia = pd.Timestamp(year=ano, month=mes, day=pd.Timestamp(year=ano, month=mes, day=1).days_in_month)

    datas_no_mes = pd.date_range(start=primeiro_dia, end=ultimo_dia)
    datas_uteis = [data for data in datas_no_mes if data.weekday() < 5]
    feriados = cal.holidays()
    datas_uteis = [data for data in datas_uteis if data not in feriados]
    return len(datas_uteis)

In [296]:
def transformar_meta_mensal_para_diaria(df):
  """Separando os dados de acordo com o nível gerente/categoria_id para que possa preencher
  com os valores da meta mensal no nível dias uteis"""

  meta_ano_gerente = []
  for ger in df['gerente_id'].unique():
    data = df[df['gerente_id']== ger]
    for cat in data['categoria_id'].unique():
      resample_ger_cat = data[data['categoria_id']==cat]
      df_diario = resample_ger_cat.reindex(dias_uteis_sem_feriado(resample_ger_cat))
      df_diario = df_diario.fillna(method='ffill')
      meta_ano_gerente.append(df_diario)
  # Concatene todos os DataFrames da lista em um único DataFrame.
  meta_concat = pd.concat(meta_ano_gerente)
  return meta_concat

In [251]:
#função para tratar os dados de 2023
#Padronizando a data
trimestre = {
    '1T': '2023-01-01',
    '2T': '2023-04-01',
    '3T': '2023-07-01',
    '4T': '2023-10-01'
}

def tratamento_dados_2023(df, id):
  #Adicionando a coluna com o gerente_id
  df['gerente_id'] = id

  #Extraindo apenas o id da categoria
  df['categoria_id'] = df['Categoria Produto'].str.extract('(\d+)')
  df.drop('Categoria Produto', axis=1, inplace=True)

  #calculando a meta trimestral
  df['1T'] = df['Previsão Vendas'] * df['1']
  df['2T'] = df['Previsão Vendas'] * df['2']
  df['3T'] = df['Previsão Vendas'] * df['3']
  df['4T'] = df['Previsão Vendas'] * df['4']

  #dropando as colunas que eu nao vou mais utilizar
  df.drop(columns=['Previsão Vendas', '1', '2', '3', '4'], axis=1, inplace=True)

  #utilizando a função metl para pivotar o dataframe
  df = pd.melt(df, id_vars=['gerente_id', 'categoria_id'], var_name='Trimestre', value_name='Valor')
  df.replace(trimestre, inplace=True)
  df['Trimestre'] = pd.to_datetime(df['Trimestre'])
  df.rename(columns={'Trimestre': 'data'}, inplace=True)
  # df.set_index('data', inplace=True)

  return df

In [252]:
def calcular_meta_diaria(df, col_meta):

  # Calcule os dias úteis para cada mês e adicione como uma nova coluna 'dias_uteis'.
  df['dias_uteis'] = df.apply(calcular_dias_uteis_no_mes, axis=1)

  # Distribua o valor de "$ Meta" igualmente pelos dias úteis.
  df['$_Meta_por_dia_util'] = round(df[col_meta] / df['dias_uteis'],2)
  df.reset_index(inplace=True)

  # filtrando as colunas que vamos utilizar e as datas do ano de 2022.
  meta_consolidado = df[['data', 'gerente_id', 'categoria_id', col_meta, '$_Meta_por_dia_util']]  #'dias_uteis',
  filtro_data = meta_consolidado['data'] >= '2022-01-01'
  meta_consolidado = meta_consolidado[filtro_data]
  return meta_consolidado


In [253]:
def calcular_meta_individual(df):
  """juntar a quantidade de funcionarios no datafrme
  das metas para calcular a meta diaria individual
  """
  meta_com_qde_func = df.merge(qde_func_por_gerente, on='gerente_id')
  meta_com_qde_func['meta_diaria_ind'] = round(meta_com_qde_func['$_Meta_por_dia_util'] / meta_com_qde_func['qde_funcionarios'],2)
  return meta_com_qde_func

In [297]:
def distribuir_meta_trimestral(df):
  """Transformar os dados mensais na granularidade diaria."""

  meta_tri = []
  for cat in df_metas_2023['categoria_id'].unique():
    filtro_cat = (df_metas_2023[df_metas_2023['categoria_id'] == cat])
    for ger in filtro_cat['gerente_id'].unique():
      filtro_ger = filtro_cat[filtro_cat['gerente_id']==ger]
      resample_cat_ger = filtro_ger.set_index('data').resample('D').ffill()
      df_diario = resample_cat_ger.reindex(dias_uteis_sem_feriado(resample_cat_ger))
      meta_tri.append(df_diario)
    #concatenando os dataframes
    meta_ger_23 = pd.concat(meta_tri)
    return meta_ger_23

de - para

In [255]:
df_de_para = (
    df_supervisor[['id', 'gerente_id']]
    .rename(columns={'id': 'supervisor_id'})
    .merge(df_vendedor[['id', 'supervisor_id']]
           .rename(columns={'id':'vendedor_id'}),
           on='supervisor_id')
    )
df_de_para

,supervisor_id,gerente_id,vendedor_id
0,1,1,12
1,2,1,1
2,2,1,4
3,2,1,11
4,3,1,6
5,3,1,7
6,4,2,2
7,4,2,5
8,4,2,8
9,5,2,3


In [256]:
#Calculando q quantidade de funcionarios por gerente
qde_func_por_gerente = df_de_para['gerente_id'].value_counts().to_frame().reset_index().rename(columns={'gerente_id':'qde_funcionarios', 'index': 'gerente_id'})
qde_func_por_gerente

,gerente_id,qde_funcionarios
0,1,6
1,2,6


#### Meta 2022

In [ ]:
#Padronizando a data
#Jan-22 deixei como dez/2021 para que possamos preencher as datas de janeiro com o método ffill
mes = {
    'Jan-22': '2021-12-31',
    'Fev-22': '2022-02-28',
    'Mar-22': '2022-03-31',
    'Abr-22': '2022-04-30',
    'Mai-22': '2022-05-31',
    'Jun-22': '2022-06-30',
    'Jul-22': '2022-07-31',
    'Ago-22': '2022-08-31',
    'Set-22': '2022-09-30',
    'Out-22': '2022-10-31',
    'Nov-22': '2022-11-30',
    'Dez-22': '2022-12-31',
}
meta22.replace(mes, inplace=True)
meta22['Mês'] = pd.to_datetime(meta22['Mês'])
meta22.rename(columns={'Mês': 'data'}, inplace=True)
meta22.set_index('data', inplace=True)

In [ ]:
meta22_concat = transformar_meta_mensal_para_diaria(meta22)

In [ ]:
# #Separando os dados de acordo com o nível gerente/categoria_id para que possa preencher com os valores da meta mensal no nível dias uteis
# meta22_gerente = []
# for ger in meta22['gerente_id'].unique():
#   data = meta22[meta22['gerente_id']== ger]
#   for cat in data['categoria_id'].unique():
#     resample_ger_cat = data[data['categoria_id']==cat]
#     df_diario = resample_ger_cat.reindex(dias_uteis_sem_feriado(resample_ger_cat))
#     df_diario = df_diario.fillna(method='ffill')
#     meta22_gerente.append(df_diario)
# # Concatene todos os DataFrames da lista em um único DataFrame.
# meta22_concat = pd.concat(meta22_gerente)
# meta22_concat

In [ ]:
meta22_consolidado = calcular_meta_diaria(meta22_concat, '$ Meta')
meta22_consolidado.head()

,data,gerente_id,categoria_id,$_Meta_por_dia_util
1,2022-01-03,1.0,1.0,1075.52
2,2022-01-04,1.0,1.0,1075.52
3,2022-01-05,1.0,1.0,1075.52
4,2022-01-06,1.0,1.0,1075.52
5,2022-01-07,1.0,1.0,1075.52


In [ ]:
# # Calcule os dias úteis para cada mês e adicione como uma nova coluna 'dias_uteis'.
# meta22_concat['dias_uteis'] = meta22_concat.apply(calcular_dias_uteis_no_mes, axis=1)


# # Distribua o valor de "$ Meta" igualmente pelos dias úteis.
# meta22_concat['$_Meta_por_dia_util'] = round(meta22_concat['$ Meta'] / meta22_concat['dias_uteis'],2)
# meta22_concat.reset_index(inplace=True)

# # filtrando as colunas que vamos utilizar e as datas do ano de 2022.
# meta22_consolidado = meta22_concat[['data', 'gerente_id', 'categoria_id',   '$_Meta_por_dia_util']]  #'dias_uteis',
# filtro_data = meta22_consolidado['data'] >= '2022-01-01'
# meta22_consolidado = meta22_consolidado[filtro_data]
# meta22_consolidado.head()


In [ ]:
meta22_com_qde_func = calcular_meta_individual(meta22_consolidado)
meta22_com_qde_func.head()

,data,gerente_id,categoria_id,$_Meta_por_dia_util,qde_funcionarios,meta_diaria_ind
0,2022-01-03,1.0,1.0,1075.52,6,179.25
1,2022-01-04,1.0,1.0,1075.52,6,179.25
2,2022-01-05,1.0,1.0,1075.52,6,179.25
3,2022-01-06,1.0,1.0,1075.52,6,179.25
4,2022-01-07,1.0,1.0,1075.52,6,179.25
...,...,...,...,...,...,...
2075,2022-12-26,2.0,4.0,787.45,6,131.24
2076,2022-12-27,2.0,4.0,787.45,6,131.24
2077,2022-12-28,2.0,4.0,787.45,6,131.24
2078,2022-12-29,2.0,4.0,787.45,6,131.24


#### Meta 2023

Gerente Victor

In [257]:
victor1_2023_1  = victor1_2023o.copy()

In [258]:
df_victor0 = tratamento_dados_2023(victor1_2023_1,1)


In [262]:
df_victor = df_victor0.copy()
df_victor.head()

,gerente_id,categoria_id,data,Valor
0,1,1,2023-01-01,30000.0
1,1,2,2023-01-01,190000.0
2,1,3,2023-01-01,30000.0
3,1,4,2023-01-01,70000.0
4,1,1,2023-04-01,120000.0


In [263]:
#Como eu nao consegui preencher o dataframe com os dados de novembro e dezembro,
# resolvi fazer na mão mesmo, criando o dataframe e concatenando com o dataframe original

faltante = [
    ['2023-11-30', 1,'1',51000.0],
    ['2023-12-31', 1,'1',51000.0],
    ['2023-11-30', 1,'2',380000.0],
    ['2023-12-31', 1,'2',380000.0],
    ['2023-11-30', 1,'3',50000.0],
    ['2023-12-31', 1,'3',50000.0],
    ['2023-11-30', 1,'4',140000.0],
    ['2023-12-31', 1,'4',140000.0],
            ]
df_faltante = pd.DataFrame(faltante, columns=['data', 'gerente_id', 'categoria_id', 'Valor'])
# Transformando as colunas 'data' e 'categoria_id' em tipo datetime e object.
df_faltante['data'] = pd.to_datetime(df_faltante['data'])
df_faltante['categoria_id'] = df_faltante['categoria_id'].astype(object)

#concatenando os dois dataframes
df_victor_c = pd.concat([df_victor, df_faltante], ignore_index=True)
df_victor_c.tail()

,gerente_id,categoria_id,data,Valor
19,1,2,2023-12-31,380000.0
20,1,3,2023-11-30,50000.0
21,1,3,2023-12-31,50000.0
22,1,4,2023-11-30,140000.0
23,1,4,2023-12-31,140000.0


Gerente Gabriel

In [260]:
gabriel2_2023_2 = gabriel2_2023o.copy()

In [261]:
df_gabriel = tratamento_dados_2023(gabriel2_2023_2,2)
df_gabriel.head()

,gerente_id,categoria_id,data,Valor
0,2,1,2023-01-01,42500.0
1,2,2,2023-01-01,150000.0
2,2,3,2023-01-01,39000.0
3,2,4,2023-01-01,40000.0
4,2,1,2023-04-01,170000.0


In [272]:
#concatenando com os dados faltantes

#Como eu nao consegui preencher o dataframe com os dados de novembro e dezembro,
# resolvi fazer na mão mesmo, criando o dataframe e concatenando com o dataframe original

faltante = [
    ['2023-11-30', 2,'1',85000.0],
    ['2023-12-31', 2,'1',85000.0],
    ['2023-11-30', 2,'2',262500.0],
    ['2023-12-31', 2,'2',262500.0],
    ['2023-11-30', 2,'3',65000.0],
    ['2023-12-31', 2,'3',65000.0],
    ['2023-11-30', 2,'4',70000.0],
    ['2023-12-31', 2,'4',70000.0],
            ]
df_faltante = pd.DataFrame(faltante, columns=['data', 'gerente_id', 'categoria_id', 'Valor'])
# Transformando as colunas 'data' e 'categoria_id' em tipo datetime e object.
df_faltante['data'] = pd.to_datetime(df_faltante['data'])
df_faltante['categoria_id'] = df_faltante['categoria_id'].astype(object)

#concatenando os dois dataframes
df_gabriel_c = pd.concat([df_gabriel, df_faltante], ignore_index=True)
df_gabriel_c.tail()

,gerente_id,categoria_id,data,Valor
19,2,2,2023-12-31,262500.0
20,2,3,2023-11-30,65000.0
21,2,3,2023-12-31,65000.0
22,2,4,2023-11-30,70000.0
23,2,4,2023-12-31,70000.0


Concatenar os dados dos dois gerentes

In [286]:
#concatenando os dados dos dois gerentes
df_metas_2023 = pd.concat([df_victor_c,df_gabriel_c ], ignore_index=True)
df_metas_2023.tail()

,gerente_id,categoria_id,data,Valor
43,2,2,2023-12-31,262500.0
44,2,3,2023-11-30,65000.0
45,2,3,2023-12-31,65000.0
46,2,4,2023-11-30,70000.0
47,2,4,2023-12-31,70000.0


In [305]:
#Distribuir as metas trimestrais
df_meta_mensal = distribuir_meta_trimestral(df_metas_2023)

In [306]:
#Calcular os dias uteis
df_meta_mensal['dias_uteis'] = df_meta_mensal.apply(calcular_dias_uteis_no_mes, axis=1)

In [307]:
#Calcular a meta diária
df_meta_diaria = calcular_meta_diaria(df_meta_mensal, 'Valor')

In [309]:
#Calcular a meta individual dos funcionarios
meta23_com_qde_func = calcular_meta_individual(df_meta_diaria)
meta23_com_qde_func.head()

,data,gerente_id,categoria_id,Valor,$_Meta_por_dia_util,qde_funcionarios,meta_diaria_ind
0,2023-01-02,1,1,30000.0,1363.64,6,227.27
1,2023-01-03,1,1,30000.0,1363.64,6,227.27
2,2023-01-04,1,1,30000.0,1363.64,6,227.27
3,2023-01-05,1,1,30000.0,1363.64,6,227.27
4,2023-01-06,1,1,30000.0,1363.64,6,227.27
